#### Importing packages

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model, save_model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier  # Import KerasClassifier
from sklearn.model_selection import cross_validate
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_validate

#### Upload dataset

In [20]:
data = pd.read_csv('../raw_data/validation_dataset_3.csv')
data.head()

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,3,0,0,1,0,0,0,0,0,0,...,1,2,0,892,0,0,0,0,0,1
1,5,0,1,3,0,3,0,2,0,0,...,1,2,1,9540,1,0,0,0,0,1
2,2,0,0,1,0,0,0,0,0,0,...,1,2,3,589,1,0,0,0,0,0
3,4,0,2,5,0,0,0,0,0,0,...,1,2,0,292,1,0,0,0,0,1
4,2,0,0,0,0,0,0,0,0,0,...,1,2,1,3597,0,1,0,0,0,0


#### Preprocessing data

##### Drop irrelevant columns

In [21]:
len(list(data.columns))

112

In [22]:
data.drop(columns=['domain_google_index', 'asn_ip'], inplace=True)

In [23]:
list(data.columns)

['qty_dot_url',
 'qty_hyphen_url',
 'qty_underline_url',
 'qty_slash_url',
 'qty_questionmark_url',
 'qty_equal_url',
 'qty_at_url',
 'qty_and_url',
 'qty_exclamation_url',
 'qty_space_url',
 'qty_tilde_url',
 'qty_comma_url',
 'qty_plus_url',
 'qty_asterisk_url',
 'qty_hashtag_url',
 'qty_dollar_url',
 'qty_percent_url',
 'qty_tld_url',
 'length_url',
 'qty_dot_domain',
 'qty_hyphen_domain',
 'qty_underline_domain',
 'qty_slash_domain',
 'qty_questionmark_domain',
 'qty_equal_domain',
 'qty_at_domain',
 'qty_and_domain',
 'qty_exclamation_domain',
 'qty_space_domain',
 'qty_tilde_domain',
 'qty_comma_domain',
 'qty_plus_domain',
 'qty_asterisk_domain',
 'qty_hashtag_domain',
 'qty_dollar_domain',
 'qty_percent_domain',
 'qty_vowels_domain',
 'domain_length',
 'domain_in_ip',
 'server_client_domain',
 'qty_dot_directory',
 'qty_hyphen_directory',
 'qty_underline_directory',
 'qty_slash_directory',
 'qty_questionmark_directory',
 'qty_equal_directory',
 'qty_at_directory',
 'qty_and_dir

##### Balance dataset

In [24]:
indices_to_remove = data[data['phishing'] == 0].index

indices_to_remove = indices_to_remove[:28000]

data.drop(indices_to_remove, inplace=True)

#### Define X and y

In [25]:
X = data.drop(columns=['phishing'])
y = data['phishing']

#### Split data on train and test sets

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42, stratify=y)

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42452, 109), (18195, 109), (42452,), (18195,))

#### Initialize the model

In [32]:
def initialize_model():
    
    reg_l1 = regularizers.L1(0.02)
    # reg_l2 = regularizers.L2(0.02)
    
    model = Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(109,)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu', bias_regularizer=reg_l1))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.00009)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

#### Fitting the model

In [34]:
model = initialize_model()

es = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

history = model.fit(X_train, y_train, 
          epochs=75, batch_size=32, 
          verbose=1, validation_data=(X_test, y_test))

Epoch 1/75
1327/1327 [==============================] - 1s 484us/step - loss: 58.5140 - accuracy: 0.6065 - val_loss: 5.6835 - val_accuracy: 0.7660
Epoch 2/75
1327/1327 [==============================] - 1s 434us/step - loss: 23.9826 - accuracy: 0.6709 - val_loss: 3.8061 - val_accuracy: 0.8076
Epoch 3/75
1327/1327 [==============================] - 1s 434us/step - loss: 12.9370 - accuracy: 0.7176 - val_loss: 1.7829 - val_accuracy: 0.8630
Epoch 4/75
1327/1327 [==============================] - 1s 436us/step - loss: 7.4597 - accuracy: 0.7613 - val_loss: 1.2293 - val_accuracy: 0.8899
Epoch 5/75
1327/1327 [==============================] - 1s 434us/step - loss: 4.4794 - accuracy: 0.8014 - val_loss: 2.5464 - val_accuracy: 0.8311
Epoch 6/75
1327/1327 [==============================] - 1s 433us/step - loss: 2.9058 - accuracy: 0.8347 - val_loss: 0.9726 - val_accuracy: 0.9053
Epoch 7/75
1327/1327 [==============================] - 1s 433us/step - loss: 2.0270 - accuracy: 0.8562 - val_loss: 0.821

#### Testing the model

In [35]:
test_accuracy = model.evaluate(X_test, y_test)[1]

569/569 [==============================] - 0s 238us/step - loss: 0.1571 - accuracy: 0.9404


#### Save the model

In [36]:
from tensorflow.keras.models import load_model, save_model

model.save(f'../saved_models/validation_model_3.h5')